**NOTE** `phimats` environment should be used as kernel

In [7]:
import numpy as np
import h5py

from PreProcessing import PhysicsConfig, MeshConfig, PreProcessing as PP

from MeshManager import MeshManager

from BoundaryConditions import *

from PostProcessing import WriteXDMF

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Simulation data

In [8]:
SimulName = "HLGB"
# Element sets
nElementSets = 1
# Simulation type
SimulType = "HLGBTrapping" 
# Number of steps to achieve the load
nSteps = 100

### Read mesh file 

The mesh file is hosted on Zenodo due to its large size.  
To download it, paste the command below into your terminal.  
**Note:** Make sure you're in the desired working directory.

```bash
wget https://zenodo.org/records/18369054/files/RVE_mesh.msh
```

In [ ]:
!wget https://zenodo.org/records/18369054/files/RVE_mesh.msh

In [9]:
# Element name
elementName = "triangle"  		# meshio compatible element name
mesh = MeshManager("RVE_mesh.msh", elementName)
mesh.WriteMesh(SimulName)

Reading mesh: RVE_100um.msh...

Mesh consistency verified: All domain elements are <triangle>.

PHIMATS MESH INITIATION
Detected 1 Material Sets. Follow this order in your Input Data:
  [Index 1] -> Physical Group: 'RVE'

------------------------------
MESH SUMMARY
------------------------------
File Name:      RVE_100um.msh
Element Type:   triangle
Dimension:      2D
Node Order:     1
Total Nodes:    1000000
Total Elements: 1996002
Cell Blocks:
  - line: 3996 items
  - triangle: 1996002 items
------------------------------



In [10]:
# Create the config object
meshConfig = MeshConfig(
    nTotNodes=mesh.get_nTotNodes(),
    nTotElements=mesh.get_nTotElements(),
    nDim=mesh.get_nDim(),
    materialNames=mesh.getMaterialNames(),
)

### Read phase-field RVE data

In [11]:
# gPhi mapped to finite element mesh

fh5 = h5py.File("Mori.rve.hdf5", "r")

try:
	gPhi_HAGB = fh5["gPhi_HAGB"][()]    
	gPhi_LAGB = fh5["gPhi_LAGB"][()]    
	fh5.close()
except:
    fh5.close()


## Diffusion and trapping data

In [12]:
# Diffusivity data
T = 300                    # Temperature [K]
R = 8.31446261815324       # Universal gas constant [J/mol.K]
dt = 10e-3  			   # Time increment [s]
N = 6          	# Number of interstitial sites per reference lattice atom [tetrahedral sites in BCC]

D01 = 8.45e-8; DQ1 = 5000  # Diffusivity m²/s
DL = D01*np.exp(-DQ1/(R*T))

H_HAGB = 15000 # HAGB enrichment enthalpy[J/mol]
H_LAGB = 10000 # LAGB enrichment enthalpy[J/mol]

Vm = 7.09e-6  # Molar volume of Fe [m³/mol]
Vgb = 2*Vm    # Molar volume around GBs  [m³/mol]

kappa_HAGB = 4*N*H_HAGB/Vm # HAGBGB occupancy enrichment ratio
KHAGB = np.exp(0.25*kappa_HAGB*Vm/(N*R*T))
print("KHAGB: ", KHAGB)

kappa_LAGB = 4*N*H_LAGB/Vm # LAGB occupancy enrichment ratio
KLAGB = np.exp(0.25*kappa_LAGB*Vm/(N*R*T))
print("KLAGB: ", KLAGB)

theta_b = 3.453e-3*Vm/N
print(theta_b)

cL = theta_b*N/Vm  # Concentration  [mol/m³]
print("cL", cL)

theta_HAGB = theta_b*KHAGB # Occupancy at the HAGB
theta_LAGB = theta_b*KLAGB # Occupancy at the LAGB

c_HAGB = theta_HAGB/Vgb
print("c_HAGB", c_HAGB)
Z_HAGB = c_HAGB/cL # Concentration enrichment ratio
print("Z_HAGB", Z_HAGB)
zeta_HAGB = 4*R*T*np.log(Z_HAGB)
print("zeta_HAGB", zeta_HAGB)

c_LAGB = theta_LAGB/Vgb
print("c_LAGB", c_LAGB)
Z_LAGB = c_LAGB/cL # Concentration enrichment ratio
print("Z_LAGB", Z_LAGB)
zeta_LAGB = 4*R*T*np.log(Z_LAGB)
print("zeta_LAGB", zeta_LAGB)

print("Eq Con HAGB: ", np.exp(0.25*zeta_HAGB/(R*T)))
print("Eq Con LAGB: ", np.exp(0.33*zeta_LAGB/(R*T)))

KHAGB:  408.9601637423769
KLAGB:  55.09607620783739
4.080295e-09
cL 0.003453
c_HAGB 0.11767828711686895
Z_HAGB 34.08001364519807
zeta_HAGB 35207.20386088472
c_LAGB 0.01585389592880521
Z_LAGB 4.591339683986449
zeta_LAGB 15207.20386088472
Eq Con HAGB:  34.08001364519806
Eq Con LAGB:  7.477562307558887


In [13]:
# Material parameters dict

Materials = {
    "RVE" : {
	"D0x1" : D01,
 	"D0y1" : D01,
	"DQx1" : DQ1,
	"DQy1" : DQ1,
 	"zeta_HAGB"  : zeta_HAGB,
  	"zeta_LAGB"  : zeta_LAGB,
	}
}

Materials

{'RVE': {'D0x1': 8.45e-08,
  'D0y1': 8.45e-08,
  'DQx1': 5000,
  'DQy1': 5000,
  'zeta_HAGB': np.float64(35207.20386088472),
  'zeta_LAGB': np.float64(15207.20386088472)}}

### Apply boundary conditions

In [55]:
# Dirichlet BCs list

exitNods = mesh.getNodesByGroup("right")
conBCs = []

# Top con
conBCs.extend(AssignDirichletBC(mesh.getNodesByGroup("left"), dof=0, value=cL))

# Bottom con
conBCs.extend(AssignDirichletBC(mesh.getNodesByGroup("right"), dof=0, value=0))

# Equilibrium boundary conditions
for i in range(len(conBCs)):
    conBCs[i][2] = conBCs[i][2]*np.exp((gPhi_HAGB[conBCs[i][0]]*zeta_HAGB+gPhi_LAGB[conBCs[i][0]]*zeta_LAGB)/(R*T))

# Write external 
WriteBCVTK(SimulName, mesh, conBCs, dofNames=['con'])

BC visualization written to HLGB_BC.vtu


In [57]:
conBCs = [[row[0], row[-1]] for row in conBCs]

### Initialize simulation object

In [58]:
# Create the config object
diffPhysConfig = PhysicsConfig(
    SimulName = SimulName,
    PhysicsType = "Transport",
    PhysicsCategory = SimulType,
    nSteps    = nSteps,
    presBCs=conBCs,
    exitNodes = exitNods,
    dt = dt,
    Temperature = T,
)

In [59]:
DiffHLGB = PP(diffPhysConfig, meshConfig, Materials)


PHIMATS: HLGB | Transport (HLGBTrapping)
  Nodes/Elems : 1000000 / 1996002
  Total DOFs  : 1000000
  BCs Count   : 2000



In [60]:
DiffHLGB.WriteInputFile()

  Input file initialized: HLGB.diff.in.hdf5


In [61]:
DiffHLGB.WriteOutputFile(OVERWRITE=True, AVCON=True, FLUX=True, AVFLUX=True)

  Output file initialized: HLGB.diff.out.hdf5


### Generate xdmf visualization file

In [62]:
WriteXDMF(SimulName, # Base simulation name
          elementName, # Element name
          nSteps+1, # Number of steps 
          components=["diff"],
		  nDim=2,
		  # mechModel="Plastic", # Adds strain_p, stress_eq, etc.
		  FLUX=True)